# EdgeTTS

https://github.com/rany2/edge-tts

edge-tts is a Python module that allows you to use Microsoft Edge's online text-to-speech service from within your Python code or using the provided edge-tts or edge-playback command.

In [ ]:
%pip install edge-tts pygame

In [42]:
import edge_tts
import os
import pygame
import time

async def generate_edge_tts_audio(text, file_name, voice='en-US-GuyNeural', style='newscast-formal', verbose=False, play=False, overwrite=False):
    communicate = edge_tts.Communicate(text, voice)
    # whether file exists?
    if os.path.exists(file_name):
        if overwrite:
            if verbose:
                print(f'{file_name} exists, overwriting...')
        else:
            if verbose:
                print(f'{file_name} exists, skipping...')
            return
        
    await communicate.save(file_name)
    if play:
        pygame.mixer.init()
        pygame.mixer.music.load(file_name)
        pygame.mixer.music.play()
    if verbose:
        print(f'{file_name} created')
    
    time.sleep(1.5)



voices = ["en-US-GuyNeural", "en-US-AriaNeural", "en-GB-RyanNeural", "en-GB-LibbyNeural"]
regions = ['us', 'us', 'uk', 'uk']
genders = ['male', 'female', 'male', 'female']

# only_us = False
only_us = True
if only_us:
    voices = voices[:2]
    print(voices)

words = """
sam
"""

for word in words.strip().split(','):
    print(word)
    for i, voice in enumerate(voices):
        w = word.strip().lower()
        if len(w) > 0:
            filename = f'../audios/{w.replace(" ", "-")}-{regions[i]}-{genders[i]}.mp3'
            await generate_edge_tts_audio(w, filename, voice=voice, verbose=True, overwrite=False, play=True)


['en-US-GuyNeural', 'en-US-AriaNeural']
ma
../audios/ma-us-male.mp3 created
../audios/ma-us-female.mp3 created

room
../audios/room-us-male.mp3 created
../audios/room-us-female.mp3 created

conversation
../audios/conversation-us-male.mp3 created
../audios/conversation-us-female.mp3 created

army
../audios/army-us-male.mp3 created
../audios/army-us-female.mp3 created

mob
../audios/mob-us-male.mp3 created
../audios/mob-us-female.mp3 created

mom
../audios/mom-us-male.mp3 created
../audios/mom-us-female.mp3 created

mind
../audios/mind-us-male.mp3 created
../audios/mind-us-female.mp3 created

night
../audios/night-us-male.mp3 created
../audios/night-us-female.mp3 created

nine
../audios/nine-us-male.mp3 created
../audios/nine-us-female.mp3 created

know
../audios/know-us-male.mp3 created
../audios/know-us-female.mp3 created

knight
../audios/knight-us-male.mp3 created
../audios/knight-us-female.mp3 created

gnaw
../audios/gnaw-us-male.mp3 created
../audios/gnaw-us-female.mp3 created

gno

# English Voices

* voice = "en-US-GuyNeural" (Male)
* voice = "en-US-AnaNeural" (Female)
* voice = "en-US-AndrewNeural" (Male)
* voice = "en-US-AriaNeural" (Female)
* voice = "en-US-AvaNeural" (Female)
* voice = "en-US-BrianNeural" (Male)
* voice = "en-US-ChristopherNeural" (Male)
* voice = "en-US-EmmaNeural" (Female)
* voice = "en-US-EricNeural" (Male)
* voice = "en-US-GuyNeural" (Male)
* voice = "en-US-JennyNeural" (Female)
* voice = "en-US-MichelleNeural" (Female)
* voice = "en-US-RogerNeural" (Male)
* voice = "en-US-SteffanNeural" (Male)
* voice = "en-GB-LibbyNeural" (Female)
* voice = "en-GB-MaisieNeural" (Female)
* voice = "en-GB-RyanNeural" (Male)
* voice = "en-GB-SoniaNeural" (Female)
* voice = "en-GB-ThomasNeural" (Male)
* voice = "en-AU-NatashaNeural" (Female)
* voice = "en-AU-WilliamNeural" (Male)
* voice = "en-CA-ClaraNeural" (Female)
* voice = "en-CA-LiamNeural" (Male)